In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [2]:
df_out = pd.read_excel("sample_input.xlsx")
df_out

,Company,Website,Address,Phone,Zip
0,"Delkor Systems, Inc",www.delkorsystems.com,4300 Round Lake Road W,(800) 328-5558,55112
1,"Liquid Packaging Solutions, Inc",www.liquidpackagingsolution.com,"3999 E. Hupp Road, R-43",(219) 393-3600,46350
2,Great Western Manufacturing,www.gwmfg.com,2017 So. 4th Street,(913) 682-2291,66048
3,Raque Food Systems LLC,www.raque.com,P.O. Box 99594,(502) 267-9641,40269
4,GK Machine Inc,www.gkmachine.com,10590 Donald Road NE,(877) 678-5525,97020
5,Blue Ridge Optics,www.blueridgeoptics.com,1617 Longwood Avenue,(540) 586-8526,24523


In [3]:
df_out.dtypes

Company    object
Website    object
Address    object
Phone      object
Zip         int64
dtype: object

In [4]:
df_out.Zip = df_out.Zip.astype("str")

In [5]:
df = pd.read_excel("sample_output.xlsx")
df

,Company,Website,Address,Phone,Zip
0,"Delkor Systems, Inc",www.delkorsystems.com,NaN,NaN,NaN
1,"Liquid Packaging Solutions, Inc",www.liquidpackagingsolution.com,NaN,NaN,NaN
2,Great Western Manufacturing,www.gwmfg.com,NaN,NaN,NaN
3,Raque Food Systems LLC,www.raque.com,NaN,NaN,NaN
4,GK Machine Inc,www.gkmachine.com,NaN,NaN,NaN
5,Blue Ridge Optics,www.blueridgeoptics.com,NaN,NaN,NaN


In [6]:
# altnerative company name
df["Company 2"] = df["Company"].replace(',','', regex=True)
df["Company 2"] = [x.rsplit(" ",1)[0] if ("llc" in x.lower() or "inc" in x.lower()) else x for x in df["Company 2"]]
df.head(4)

,Company,Website,Address,Phone,Zip,Company 2
0,"Delkor Systems, Inc",www.delkorsystems.com,NaN,NaN,NaN,Delkor Systems
1,"Liquid Packaging Solutions, Inc",www.liquidpackagingsolution.com,NaN,NaN,NaN,Liquid Packaging Solutions
2,Great Western Manufacturing,www.gwmfg.com,NaN,NaN,NaN,Great Western Manufacturing
3,Raque Food Systems LLC,www.raque.com,NaN,NaN,NaN,Raque Food Systems


In [7]:
def googleSearch(query):
    with requests.session() as c:
        url = 'https://www.google.com/search'
        query = {'q': query}
        urllink = requests.get(url, params=query)
        return urllink.url

In [8]:
df["HTML"] = [BeautifulSoup(requests.get(googleSearch(x)).content, 'html.parser')\
                 .find(text='Address').parent.parent.parent.parent.parent\
                 if type(BeautifulSoup(requests.get(googleSearch(x)).content, 'html.parser')\
                 .find(text='Address')) != type(None) else None\
                 for x in df['Company']]

In [9]:
df["HTML 2"] = [BeautifulSoup(requests.get(googleSearch(x)).content, 'html.parser')\
                 .find(text='Address').parent.parent.parent.parent.parent\
                 if type(BeautifulSoup(requests.get(googleSearch(x)).content, 'html.parser')\
                 .find(text='Address')) != type(None) else None\
                 for x in df['Company 2']]

In [10]:
df["Address HTML"] = df["HTML"].apply(lambda x: x.contents[0].contents[0].contents[2].select_one('span').text.strip()\
                                     if type(x) != type(None) else None)
df["Address HTML 2"] = df["HTML 2"].apply(lambda x: x.contents[0].contents[0].contents[2].select_one('span').text.strip()\
                                     if type(x) != type(None) else None)

In [11]:
df["Phone HTML"] = df["HTML"].apply(lambda x: x.contents[2].contents[0].contents[2].select_one('span').text.strip()\
                                     if type(x) != type(None) else None)
df["Phone HTML 2"] = df["HTML 2"].apply(lambda x: x.contents[2].contents[0].contents[2].select_one('span').text.strip()\
                                     if type(x) != type(None) else None)

In [12]:
df['Phone'] = df['Phone HTML'].combine_first(df['Phone HTML 2'])
df['Address'] = df['Address HTML'].combine_first(df['Address HTML 2'])

df["Zip"] = [x.rsplit(" ",1)[1].strip() for x in df["Address"]]
df["Address"] = [x.split(",",1)[0] for x in df["Address"]]

df.drop(['HTML', 'HTML 2', 'Phone HTML', 'Phone HTML 2', 'Address HTML', 'Address HTML 2', 'Company 2'], \
        axis=1, inplace=True)

In [13]:
df == df_out

,Company,Website,Address,Phone,Zip
0,True,True,False,True,True
1,True,True,False,True,True
2,True,True,False,True,True
3,True,True,False,True,False
4,True,True,False,False,True
5,True,True,False,False,True


In [15]:
df.to_csv('df.csv')

In [14]:
# BeautifulSoup(requests.get(googleSearch(x)).content, 'html.parser').find(text='Website').parent.parent